In [271]:
import tensorflow as tf
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

In [272]:
header = ['user_id','item_id','rating','timestamp']
df = pd.read_csv("ml-100k/u.data",sep = '\t',names = header)
#  读取 100000 行 user -item -rating 的数据


In [273]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(n_users,n_items)

943 1682


In [288]:
# 按 90% 训练集 10% 的测试集 拆分数据
train_data,test_data = map (np.array, train_test_split(df,test_size = 0.2))

In [320]:
embedding_size = 10
batch_size = 200
# 搭建NN
user_id = tf.placeholder(tf.int32,[batch_size])
item_id = tf.placeholder(tf.int32,[batch_size])
rate = tf.placeholder(tf.float64,[batch_size,1])
keep_prob = tf.placeholder(tf.float32)

In [321]:
user_embeding = tf.Variable(tf.random_uniform([n_users+1,\
                        embedding_size]),trainable = True)

item_embeding = tf.Variable(tf.random_uniform([n_items+1,\
                        embedding_size]),trainable = True)



In [322]:
user_input = tf.nn.embedding_lookup(user_embeding,user_id)
item_input = tf.nn.embedding_lookup(item_embeding,item_id)


In [323]:
fc_input = tf.multiply(user_input,item_input)
# 我们定义 第一层 100 个神经元  200 * 10 * 10 *100 + 【100】 = 200 * 100
# w_1 初始值 按高斯分布去
# bias 初始值为 0
w_1 = tf.Variable(tf.truncated_normal([embedding_size,100],stddev=0.1))
b_1 = tf.Variable(tf.constant(0.,shape=[100]))
l_1 = tf.nn.relu(tf.nn.xw_plus_b(fc_input,w_1,b_1))
# 以keep_prob的概率 扔掉 神经元
l_1_drop = tf.nn.dropout(l_1,keep_prob)

In [324]:
# 第二层 50个神经元  200 *100 * 100 *50 + 【50】 = 200 * 50
w_2 = tf.Variable(tf.truncated_normal([100,50],stddev=0.1))
b_2 = tf.Variable(tf.constant(0.,shape=[50]))
l_2 = tf.nn.relu(tf.nn.xw_plus_b(l_1_drop,w_2,b_2))
l_2_drop = tf.nn.dropout(l_2,keep_prob)

In [325]:
# 输出层 一个神经元 200 * 50 * 50 * 1 + 【1】 = 200 *1 
w_3 = tf.Variable(tf.truncated_normal([50,1],stddev=0.1))
b_3 = tf.Variable(tf.constant(0.,shape =[1]))
prediction = tf.nn.relu(tf.nn.xw_plus_b(l_2_drop,w_3,b_3))

In [326]:
# 使用 MSE 
loss = tf.losses.mean_squared_error(rate,prediction)
# Adam优化 学习率 0.001
train = tf.train.AdamOptimizer(0.001).minimize(loss)

In [327]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("start learning")
    for epoch in range(10):
        print("epoch:{}".format(epoch))
        train_loss = []
        for start,end in zip(range(0,len(train_data),batch_size),range(batch_size,len(train_data),batch_size)):
            tr_loss,_ = sess.run([loss, train], feed_dict={user_id : train_data[start:end,0], item_id : train_data[start:end,1], rate: train_data[start:end,2].reshape(batch_size,1), keep_prob: 0.5})
            train_loss.append(tr_loss)
        # test 
        rmse=[]
        for start,end in zip(range(0,len(test_data),batch_size),range(batch_size,len(test_data),batch_size)):
            pred = sess.run(prediction, feed_dict={user_id : test_data[start:end,0], item_id : test_data[start:end,1], keep_prob: 1.0})
            pred=np.array([ [min(max(i[0],1),5)] for i in pred])
            rmse.append((pred - test_data[start:end,2].reshape(batch_size,1))**2)
        print(np.mean(rmse))
        print("loss: {:.3f}, rmse:{:.3f}".format(np.mean(train_loss),np.sqrt(np.mean(rmse))))


start learning
epoch:0
1.2214755462973774
loss: 2.761, rmse:1.105
epoch:1
1.112986017795254
loss: 1.596, rmse:1.055
epoch:2
1.0301056700581321
loss: 1.431, rmse:1.015
epoch:3
0.9745297240938611
loss: 1.309, rmse:0.987
epoch:4
0.9580527728470296
loss: 1.233, rmse:0.979
epoch:5
0.9451448430011485
loss: 1.172, rmse:0.972
epoch:6
0.9369008225011338
loss: 1.122, rmse:0.968
epoch:7
0.9380395396859423
loss: 1.083, rmse:0.969
epoch:8
0.9272159787774171
loss: 1.043, rmse:0.963
epoch:9
0.9248870234397868
loss: 1.013, rmse:0.962
